In [7]:
import os
import copy
import difflib
import shutil
from glob import glob
import pandas as pd
import numpy as np
import operator
import sys

from mido import MidiFile
import pretty_midi as pm
import music21

sys.path.append('/media/manu/Data/PycharmProjects/thesis')

from utils import get_chord_progressions, calculate_melody_results, flatten_chord_progression
from melody import Melody

source = 'JazzPage'

In [8]:
input_folder = f'../data/Complete Examples/v1.2/{source}'
filepaths = [y for x in os.walk(input_folder) for y in glob(os.path.join(x[0], '*.mid'))]

auto_folder  = f'../data/Complete Examples Melodies Auto/v1.2/{source}'
auto_filenames = [os.path.basename(y) for x in os.walk(auto_folder) for y in glob(os.path.join(x[0], '*.mid'))]

manual_folder  = f'../data/Complete Examples Melodies Manual/v1.2/{source}'
manual_filenames = [os.path.basename(y) for x in os.walk(manual_folder) for y in glob(os.path.join(x[0], '*.mid'))]

print(len(filepaths))
filepaths = [f for f in filepaths 
             if os.path.basename(f) not in manual_filenames 
             and os.path.basename(f) not in auto_filenames]
print(len(filepaths))

92
29


In [9]:
idx = 0
file = filepaths[idx]
print(file)

../data/Complete Examples/v1.2/JazzPage/Ain't Misbehavin'.mid


In [10]:
filepaths[idx:]

["../data/Complete Examples/v1.2/JazzPage/Ain't Misbehavin'.mid",
 '../data/Complete Examples/v1.2/JazzPage/Anthropology.mid',
 '../data/Complete Examples/v1.2/JazzPage/Autumn Leaves (2).mid',
 '../data/Complete Examples/v1.2/JazzPage/Autumn Leaves (3).mid',
 '../data/Complete Examples/v1.2/JazzPage/Autumn Leaves.mid',
 '../data/Complete Examples/v1.2/JazzPage/Blue Monk (2).mid',
 '../data/Complete Examples/v1.2/JazzPage/Blue Train.mid',
 '../data/Complete Examples/v1.2/JazzPage/Christmas Song (1).mid',
 '../data/Complete Examples/v1.2/JazzPage/Criss-Cross.mid',
 '../data/Complete Examples/v1.2/JazzPage/Doxy.mid',
 '../data/Complete Examples/v1.2/JazzPage/How Insensitive.mid',
 '../data/Complete Examples/v1.2/JazzPage/I Concentrate On You (2).mid',
 '../data/Complete Examples/v1.2/JazzPage/I Got Rhythm.mid',
 "../data/Complete Examples/v1.2/JazzPage/I Won't Dance.mid",
 "../data/Complete Examples/v1.2/JazzPage/It Don't Mean A Thing.mid",
 "../data/Complete Examples/v1.2/JazzPage/It's A

In [5]:
pm_m = pm.PrettyMIDI(file)

In [6]:
for idx, instr in enumerate(pm_m.instruments):
    if instr.is_drum:
        print(f'{idx} - Drums')
    else:
        print(f'{idx} - {pm.utilities.program_to_instrument_name(instr.program):25} - {instr.name}')

0 - Electric Guitar (jazz)    - 
1 - Electric Guitar (jazz)    - 


In [7]:
melody_idx = 0
melody_track = pm_m.instruments[melody_idx]

out_path = os.path.join('..', 'data', 'Complete Examples Melodies Manual', f'v1.2')
if not os.path.exists(os.path.join(out_path, source)):
    os.makedirs(os.path.join(out_path, source))

out_filename = os.path.join(out_path, source, os.path.basename(file))
out_filename

'../data/Complete Examples Melodies Manual/v1.2/Jazz-Midi/Solar.mid'

In [8]:
# out_filename = out_filename.replace('.mid', '') + ' (2).mid'

pm_melody = copy.deepcopy(pm_m)
pm_melody.instruments = [melody_track]

pm_melody.write(out_filename)

In [26]:
def get_channels(m):
    channels = set({})
    for t in m.tracks:
        for m in t:
            if m.type == 'note_on':
                channels.add(m.channel)
                    
    return channels

In [27]:
def get_programs_mido(m, channels):
    programs = []
    
    for c in channels:
        programs.append(tuple([0, c]))
    
    for t in m.tracks:
        for m in t:
            if m.type == 'program_change':
                programs.append(tuple([m.program, m.channel]))
                
    return programs

In [28]:
def get_programs_pm(m):
    programs = []
    
    for i in m.instruments:
        programs.append(tuple([i.program, i.is_drum]))
        
    return programs

In [29]:
def filter_instruments(pm_p):
    filtered_instr = [
        i for i in pm_p.instruments 
        if i.is_drum == False
        and all(
            sub not in i.name.lower() for sub in non_melody_names
        )]
    
    return filtered_instr

In [30]:
def get_melody_tracks(pm_p):
    melody_tracks = [i for i in pm_p.instruments if 'solo' in i.name.lower()]
    
    if len(melody_tracks) == 0:
        melody_tracks = [
            i for i in pm_p.instruments
            if any(
                sub in i.name.lower() for sub in melody_names
            )]
    
    return melody_tracks

In [34]:
melody_names = ['melody', 'melodia', 'melodía', 'lead']
non_melody_names = ['bass', 'bajo', 'basso', 'baixo', 'drum', 'percussion', 'batería', 'bateria', 'chord', 'rhythm',
                     'cymbal', 'clap', 'kick', 'snare', 'hh ', 'hats', 'ride', 'kit']

'Music Box'